AI-Powered Smart Text Analyzer:
A cutting-edge NLP tool that summarizes, paraphrases, analyzes sentiment, extracts keywords, classifies topics, evaluates readability, and generates questions—all in one place!

Loading the PEGASUS Model and Tokenizer


In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

Initializing the PEGASUS Summarization Model

In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# Load model with low RAM usage
MODEL_NAME = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(MODEL_NAME)
model = PegasusForConditionalGeneration.from_pretrained(MODEL_NAME, torch_dtype=torch.float16)
model.to("cpu")  # Use "cuda" if you have GPU


Text Summarization Function

In [ ]:
def summarize_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=80,  # Keeps summary concise
            min_length=50,  # Prevents overly short output
            length_penalty=1.3,  # Makes sure it doesn't favor longer phrases
            num_beams=7,  # Slightly higher for better diversity
            no_repeat_ngram_size=3,  # Prevents repetition
            early_stopping=True,
            repetition_penalty=1.15  # Avoids over-favoring specific parts
        )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


 Generating a Summary with PEGASUS

In [ ]:
text = """Artificial Intelligence (AI) is transforming industries by automating tasks,
improving efficiency, and generating insights. From healthcare to finance, AI-powered solutions
are revolutionizing the way we work and live. However, ethical concerns around data privacy
and algorithmic bias remain significant challenges in AI development. Researchers and policymakers
are working together to ensure AI benefits society while minimizing risks."""

summary = summarize_text(text)
print("Summarized Text:\n", summary)


In [ ]:
!pip install yake

 Keyword Extraction using YAKE

In [ ]:
import yake

def extract_keywords(text, num_keywords=5):
    """Extract important keywords from the text."""
    kw_extractor = yake.KeywordExtractor()
    keywords = kw_extractor.extract_keywords(text)
    return [kw[0] for kw in keywords[:num_keywords]]

# Example usage
keywords = extract_keywords(text)
print("Keywords:", keywords)


Sentiment Analysis using TextBlob


In [ ]:
from textblob import TextBlob

def analyze_sentiment(text):
    """Determine sentiment polarity of the text."""
    sentiment = TextBlob(text).sentiment.polarity
    if sentiment > 0:
        return "Positive"
    elif sentiment < 0:
        return "Negative"
    else:
        return "Neutral"

# Example usage
sentiment = analyze_sentiment(text)
print("Sentiment:", sentiment)


In [ ]:
pip install transformers


 Topic Classification using Zero-Shot Learning

In [ ]:
from transformers import pipeline
topic_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def classify_topic(text):
    labels = ["Technology", "Health", "Finance", "Education", "Entertainment", "Politics", "Sports"]
    result = topic_classifier(text, candidate_labels=labels)
    return result["labels"][0]  # Most relevant topic

topic = classify_topic(text)
print("Detected Topic:", topic)


 Text Paraphrasing using T5 Model

In [ ]:
from transformers import pipeline

# Load paraphrasing model
paraphraser = pipeline("text2text-generation", model="t5-small")

def paraphrase_text(text):

    result = paraphraser(f"paraphrase: {text}", max_length=100, do_sample=False)
    return result[0]['generated_text']

# Example usage
paraphrased = paraphrase_text("AI is changing the world by automating tasks.")
print("Paraphrased Text:", paraphrased)


In [ ]:
!pip install textstat

Readability Analysis using TextStat


In [ ]:
import textstat

def get_readability_score(text):
    """Get Flesch reading ease score."""
    score = textstat.flesch_reading_ease(text)
    return f"Readability Score: {score} (Higher = Easier to Read)"


score = get_readability_score(text)
print(score)


Automatic Question Generation using T5 Model

In [ ]:
question_generator = pipeline("text2text-generation", model="t5-large")

def generate_questions(summary):
    """Generate a better question from the summary."""
    prompt = f"Generate a question based on this text: {summary}"
    result = question_generator(prompt, max_length=50, do_sample=True)
    return result[0]['generated_text']

# Example usage

question = generate_questions(text)
print("Generated Question:", question)


Visualizing Topic Classification Results

In [ ]:
import matplotlib.pyplot as plt

def plot_topic_distribution(topic):
    """Display topic classification results as a pie chart."""
    topic_data = {topic: 1}  # Only one topic per input
    plt.pie(topic_data.values(), labels=topic_data.keys(), autopct="%1.1f%%", colors=["skyblue"])
    plt.title("Detected Topic Distribution")
    plt.show()

# Example usage
plot_topic_distribution(topic)


Evaluating Summary Quality with ROUGE Score

In [ ]:
from rouge_score import rouge_scorer

def evaluate_summary(reference, generated):
    """Calculate ROUGE score between reference text and generated summary."""
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = scorer.score(reference, generated)

    for key, value in scores.items():
        print(f"{key}: Precision={value.precision:.4f}, Recall={value.recall:.4f}, F1-Score={value.fmeasure:.4f}")

# Example usage
evaluate_summary(text, summary)
